### Import libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time
import statistics
import glob
import math
import os
sns.set(style="whitegrid")

In [ ]:
from utilityFuncs import Catch22
from utilityFuncs import Kats
from utilityFuncs import TSFeatures
from utilityFuncs import TSFel
from utilityFuncs import TSFresh

--------------------------------------

### Load time series

Load the generated time series of the folder _TimeSeries_.

In [ ]:
timeSeriesLength = [100, 250, 500, 750, 1000]
timeSeries = []
numberOfSeries = 10

for filename in glob.glob('TimeSeries/*.txt'):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        
        # saves data in a matrix
        tsAsArray = np.loadtxt(f, dtype='float')
        
        # transform into a dataframe and add to collection
        ts= pd.DataFrame(tsAsArray)
        timeSeries.append(ts)


Since the naming is based on the length of the time series, it first loads the 1000 long ones --> needs to be changed.

In [ ]:
[timeSeries.append(timeSeries[i]) for i in range(numberOfSeries)]
[timeSeries.pop(0) for _ in range(numberOfSeries)]

--------------------------------------

### COMPUTATION TIME BY SERIES

Same base function for each feature set:

In [ ]:
# stores the number of features calculated by each feature set
nbrFeats = []

def measureTime(featureSet):
    
    medians = []

    for i in range(len(timeSeriesLength)):
    
        # the first 10 time series are of length 10, the next 10 of length 250 and so on
        part = i*numberOfSeries
    
        timeMeasures = []
    
        for j in range(numberOfSeries):
        
            ind = j + part
            ts = timeSeries[ind].values.reshape(len(timeSeries[ind]))
            
            # get the time
            start = time.time()
            f = featureSet(ts)
            end = time.time()
        
            timeMeasures.append(end-start)
        
        medians.append(statistics.median(timeMeasures))
    
    nbrFeats.append(len(f.iloc[0]))
    return medians

Get time measurements for each feature set:

In [ ]:
catch22Time    = measureTime(Catch22.calcFeats)
katsTime       = measureTime(Kats.calcFeats)
tsfeaturesTime = measureTime(TSFeatures.calcFeats)
tsfelTime      = measureTime(TSFel.calcFeats)
tsfreshTime    = measureTime(TSFresh.calcFeats)

measurments = [catch22Time,
               katsTime,
               tsfeaturesTime,
               tsfelTime,
               tsfreshTime]

### Plotting

In [ ]:
fig, ax = plt.subplots()
lw = 2.5
colorsXsets = [('catch22', 'firebrick'),
               ('kats', 'darkorange'),
               ('tsfeatures', 'gold'),
               ('tsfel', 'forestgreen'),
               ('tsfresh', 'navy')]

for y, i in zip(measurments, colorsXsets):
    ax = sns.lineplot(x = timeSeriesLength, y = y, color=i[1], label=i[0], linewidth=lw)
    
ax.set(xlabel='Time series length', ylabel='Computation time (s)')

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.set_yscale('log')
ax.set_xscale('log')

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.title('Computation time by length of time series')
plt.show()

fig.savefig('outputs/compTime1.pdf')

--------------------------------------

### COMPUTATION TIME PER FEATURE BY SERIES LENGTH

In [ ]:
fig, ax = plt.subplots()
lw = 2.5
colorsXsets = [('catch22', 'firebrick'),
               ('kats', 'darkorange'),
               ('tsfeatures', 'gold'),
               ('tsfel', 'forestgreen'),
               ('tsfresh', 'navy')]

for i in range(len(measurments)):
    ax = sns.lineplot(x = timeSeriesLength, y = np.array( measurments[i]) / nbrFeats[i],
                      color=colorsXsets[i][1], label=colorsXsets[i][0], linewidth=lw)
    
ax.set(xlabel='Time series length', ylabel='Computation time per feature (s)')

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.set_yscale('log')
ax.set_xscale('log')

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.title('Computation time per feature by length of time series')
plt.show()

fig.savefig('outputs/compTime2.pdf')